In [ ]:
from django.contrib.gis.gdal import DataSource
from helper_functions.LayerMapping2 import LayerMapping
import os
import shapefile
import re
from django.db import transaction

In [ ]:
mapping = {'name': 'name', 'geom': 'POLYGON',
           'start_date': 'start_date', 'end_date': 'end_date', 'date_accuracy': 'date_acc',
           'orig_id': 'id'}

In [ ]:
folder = '/mnt/acdh_resources/container/R_histogis_7663/mosaic/18_Österreich/austro-hungarian-empire_1910'

In [ ]:
ns, created = SkosNamespace.objects.get_or_create(namespace='https://histogis.acdh.oeaw.ac.at/vocabs/administrative_levels', prefix='hg')
scheme, created = SkosConceptScheme.objects.get_or_create(dc_title='Administrative Levels HistoGis', namespace=ns)

In [ ]:
lst_files = {}
lst_files_id = {}
src, created = Source.objects.get_or_create(name='Austria-Hungary 1910 (mosaic data)', original_url='http://www.censusmosaic.org/mosaic/tmp/download/austro-hungarian_empire_1910.zip')
for fn in os.listdir(folder):
    if fn.endswith('.shp'):
        print(fn)
        fn_short = fn.split('.')[0]
        #r = shapefile.Reader(os.path.join(folder, fn_short))
        lm = LayerMapping(TempSpatial, os.path.join(folder, fn), mapping, encoding='latin1')
        lm.save(verbose=True)
        adm = re.search('adm[0-9]+', fn_short)
        voc, created = SkosConcept.objects.get_or_create(pref_label=adm.group(0), namespace=ns)
        if created:
            voc.scheme.add(scheme)
        for obj_id in lm.saved_object_ids:
            print(obj_id)
            obj = TempSpatial.objects.get(pk=obj_id)
            obj.source = src
            obj.administrative_unit = voc
            obj.save()
            lst_files['{}|{}'.format(fn_short, obj.name)] = obj_id

In [ ]:
lst_files

In [ ]:
with transaction.atomic():    
    for fn in os.listdir(folder):
        if fn.endswith('.shp'):
            fn_short = fn.split('.')[0]
            r = shapefile.Reader(os.path.join(folder, fn_short), encoding="latin1")
            for rr in r.iterShapeRecords():
                parent = rr.record[6]
                idx = rr.record[0]
                print(parent)
                if len(parent) > 5:
                    try:
                        parent = parent.decode('latin1')
                    except:
                        pass
                    try:
                        parent2 = lst_files[parent]
                    except:
                        parent = parent.replace('ü', 'ue')
                        parent = parent.replace('ö', 'oe')
                        parent = parent.replace('ä', 'ae')
                    try:    
                        parent2 = lst_files[parent]
                    except:
                        print('key error: {}'.format(parent))
                        continue
                    t1 = TempSpatial.objects.get(pk=parent2)
                    t2 = TempSpatial.objects.get(orig_id=idx)
                    t2.part_of = t1
                    t2.save()